In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import nltk
import re
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.datasets import fetch_20newsgroups
import numpy as np

nltk.download('stopwords')
stemmer = SnowballStemmer("english")
stop_words = set(stopwords.words('english'))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
def load_dataset(a_set, cats):
    dataset = fetch_20newsgroups(subset=a_set, categories=cats,
                                remove=('headers', 'footers', 'quotes'),
                                shuffle=True)
    return dataset

categories = ["comp.windows.x", "misc.forsale", "rec.autos", "rec.motorcycles",
            "rec.sport.baseball", "rec.sport.hockey", "sci.crypt", "sci.med",
            "sci.space", "talk.politics.mideast"]

In [3]:
newsgroups_all = load_dataset('all', categories)
print(f"Loaded {len(newsgroups_all.data)} docs.")

Loaded 9850 docs.


In [4]:
def preprocess(text):
    # Usuń znaki specjalne i liczby używając wyrażenia regularnego
    text = re.sub(r'[^a-zA-Z\s]', '', text)

     # Podziel tekst na tokeny, usuwając stop words i stosując stemming
    tokens = [stemmer.stem(word) for word in text.lower().split() if word not in stop_words]
    return ' '.join(tokens)

In [5]:
preprocessed_docs = [preprocess(doc) for doc in newsgroups_all.data]

In [6]:

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF

In [7]:
# Wybierz dwa z LSA, LDA i NMF, zaimplementuj te rozwiązania i porównaj wyniki
# wyświetl 10 tematów, porównaj je ze sobą
# wyświetl artykuły (albo jego część np. 100 słów) dla jednego z tematów i zobacz co na jego temat mają do powiedzenia modele
# wyznaczamy tematy które mają być podobne dla dwóch modeli, a następnie sprawdzamy czy artykły w tych tematach są podobne

vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=1000)
tfidf = vectorizer.fit_transform(preprocessed_docs)

# LDA
lda = LatentDirichletAllocation(n_components=10, random_state=42)
lda_topics = lda.fit_transform(tfidf)

# NMF
nmf = NMF(n_components=10, random_state=42)
nmf_topics = nmf.fit_transform(tfidf)

def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic {topic_idx + 1}:")
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

# Wyświetl 10 tematów
no_top_words = 10
feature_names = vectorizer.get_feature_names_out()
print("LDA Topics:")
display_topics(lda, feature_names, no_top_words)
print("\nNMF Topics:")
display_topics(nmf, feature_names, no_top_words)


LDA Topics:
Topic 1:
dog motorcycl univers new delet april pictur lost th california
Topic 2:
game team hockey basebal fan play year player go think
Topic 3:
game player play year team hit pitch win score run
Topic 4:
key use email pleas sale offer encrypt system chip includ
Topic 5:
car bike would like one get use engin ride dont
Topic 6:
israel armenian peopl jew arab space isra us would muslim
Topic 7:
mr oh quot articl answer know want instead someth your
Topic 8:
window use server file program widget thank run display motif
Topic 9:
david van pit bos la det trust nsa think say
Topic 10:
one would dont know peopl like get effect think use

NMF Topics:
Topic 1:
like dont would get one think go know im thing
Topic 2:
team year player play season last win leagu hockey hit
Topic 3:
thank pleas email anyon know list mail post send address
Topic 4:
key chip encrypt clipper use secur govern phone escrow system
Topic 5:
armenian turkish muslim peopl genocid armenia turk turkey govern russi

In [8]:
def display_articles_for_topic(model_topics, original_data, topic_index, num_articles=3):
  """Wyświetla określoną liczbę artykułów dla danego tematu.

  Args:
    model_topics: Macierz tematów z modelu (np. lda_topics lub nmf_topics).
    original_data: Oryginalne dane tekstowe (newsgroups_all.data).
    topic_index: Indeks tematu, dla którego chcemy wyświetlić artykuły.
    num_articles: Liczba artykułów do wyświetlenia.
  """
  topic_docs = np.argsort(model_topics[:, topic_index])[::-1]
  for i in range(num_articles):
    doc_index = topic_docs[i]
    print(f"Artykuł {i + 1} dla tematu {topic_index + 1}:\n")
    print(original_data[doc_index][:200] + "...\n")

In [10]:
num_topics = 3  # Liczba tematów

# Dla modelu LDA
print("Artykuły dla tematów LDA:")
for topic_index in range(num_topics):
  display_articles_for_topic(lda_topics, newsgroups_all.data, topic_index)

# Dla modelu NMF
print("\nArtykuły dla tematów NMF:")
for topic_index in range(num_topics):
  display_articles_for_topic(nmf_topics, newsgroups_all.data, topic_index)

Artykuły dla tematów LDA:
Artykuł 1 dla tematu 1:

MLB Standings and Scores for Tuesday, April 6th, 1993
	                 (including yesterday's games)

NATIONAL WEST	      Won  Lost   Pct.    GB   Last 10   Streak    Home   Road
Atlanta Braves      ...

Artykuł 2 dla tematu 1:

NOTE: Saturday, April 20th's scores should be sent out by this coming Friday.

	     MLB Standings and Scores for Tuesday, April 20th, 1993
	                   (including yesterday's games)

NATIONAL ...

Artykuł 3 dla tematu 1:

MLB Standings and Scores for Friday, April 16th, 1993
	                   (including yesterday's games)

NATIONAL WEST	      Won  Lost   Pct.    GB   Last 10  Streak    Home   Road
San Francisco Giant...

Artykuł 1 dla tematu 2:


Hindsight's 20-20. Nobody expected Kisio to have his 2nd best career year.


I didn't mean to imply that Gund's offer wasn't enough.  Gund's offer
was the right $, but too late in the season.  $2M of...

Artykuł 2 dla tematu 2:

I hope that this comes off as

In [11]:
topic_index = 1

# Dla modelu LDA
print("Artykuły dla tematu 2 w modelu LDA:")
display_articles_for_topic(lda_topics, newsgroups_all.data, topic_index)

# Dla modelu NMF
print("\nArtykuły dla tematu 2 w modelu NMF:")
display_articles_for_topic(nmf_topics, newsgroups_all.data, topic_index)

Artykuły dla tematu 2 w modelu LDA:
Artykuł 1 dla tematu 2:


Hindsight's 20-20. Nobody expected Kisio to have his 2nd best career year.


I didn't mean to imply that Gund's offer wasn't enough.  Gund's offer
was the right $, but too late in the season.  $2M of...

Artykuł 2 dla tematu 2:

I hope that this comes off as a somewhat unbiased assesment
of WFAN and WIP(I go to school in Philadelphia, and I listen to
both stations on a consistant basis.)  Now that the fan has Mike
Lupica on f...

Artykuł 3 dla tematu 2:

 
 
 Well, here in Detroit we get to see and hear Don Cherry quite regularly.
The Detroit area picks up Hockey Night in Canada from CBC (ch 9) in
Windsor where we see Don Cherry's Coaches Corner betwe...


Artykuły dla tematu 2 w modelu NMF:
Artykuł 1 dla tematu 2:



"Deeply rooted rivalry?" Ahem, Jokerit have been around since 1967 and joined
the top flight only in the early '70s. Helsingfors IFK have been around since
1897 but fans only started taking hockey s...

Artykuł